In [1]:
# Use autoreload to automatically reload modules
%load_ext autoreload
%autoreload 2

In [2]:
import rootutils
root_path = rootutils.find_root()

In [21]:
import numpy as np

# import npy file
meta = np.load(root_path / "data/processed/embeddings/africa/positive/event_0/metadata.npy", allow_pickle=True)
meta

array({'event_id': 0, 'centroid_lat': -7.642132, 'centroid_lon': 14.956888, 'num_timesteps': 2, 'timestamp_before': '2020-08-20T17:24:39.583177000', 'timestamp_after': '2020-09-25T17:24:41.193407000', 'earliest_alert': '2020-09-01', 'latest_alert': '2020-09-25', 'duration_days': 24, 'sample_type': 'positive', 'continent': 'africa', 'model_mode': 's1_only', 'size_pixels': 123},
      dtype=object)

In [3]:
import cubo
import xarray as xr
# import basic libraries
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
import numpy as np

In [4]:
from burned_embedder import utils

In [6]:
import pandas as pd

In [9]:
pd.read_parquet(root_path / "data/processed/radd/southeast_asia_negative_clean.parquet")

,centroid_y,centroid_x,forest_fraction,earliest_alert,latest_alert,safety_buffer_months,is_deforestation,attempt_number,positive_event_id,positive_centroid_y,positive_centroid_x,positive_area_hectares,positive_duration_days,tile_name,geometry
0,-6.39395,138.71125,1.000000,2020-09-24,2020-10-19,2,False,4,1,-0.917807,133.797630,2.020644,25,00N_130E,b'\x01\x01\x00\x00\x00\xf6(\\\x8f\xc2Va@l\t\xf...
1,-1.74665,136.24165,0.988453,2025-07-27,2025-08-24,2,False,5,2,-3.615511,137.712052,1.269063,28,00N_130E,b'\x01\x01\x00\x00\x00\x82\xe2\xc7\x98\xbb\x07...
2,-4.04735,138.24485,0.891819,2023-08-10,2023-09-03,2,False,15,3,-7.711489,131.300387,3.338991,24,00N_130E,b'\x01\x01\x00\x00\x00\x9e\xcd\xaa\xcf\xd5Ga@\...
3,-3.08245,139.31805,0.699903,2023-11-02,2023-11-14,2,False,11,4,-7.756452,131.188611,3.449880,12,00N_130E,b'\x01\x01\x00\x00\x00\xc5\x8f1w-ja@\rq\xac\x8...
4,-9.97855,148.43325,1.000000,2025-08-17,2025-09-10,2,False,1,5,-5.196355,142.993613,1.946718,24,00N_140E,b'\x01\x01\x00\x00\x00\xbe\x9f\x1a/\xdd\x8db@i...
5,-9.62025,148.31885,0.809641,2023-11-01,2023-11-13,2,False,1,6,-7.930921,140.134958,4.250745,12,00N_140E,b'\x01\x01\x00\x00\x00\x8cJ\xea\x044\x8ab@!\xb...
6,-9.61165,149.22645,1.000000,2020-08-21,2020-09-14,2,False,11,7,-7.951727,140.239191,2.242422,24,00N_140E,b'\x01\x01\x00\x00\x00\xbc\x05\x12\x14?\xa7b@b...
7,-4.57905,141.01175,0.830238,2024-04-26,2024-05-23,2,False,3,9,-6.906919,140.926448,1.306026,27,00N_140E,b'\x01\x01\x00\x00\x00L7\x89A`\xa0a@\xc6\xfe\x...
8,-5.62405,145.75955,0.958706,2025-08-18,2025-09-07,2,False,7,10,-5.841183,149.493737,1.983681,20,00N_140E,b'\x01\x01\x00\x00\x00\xa85\xcd;N8b@tF\x94\xf6...
9,-8.87575,148.00405,0.978016,2025-08-02,2025-08-26,2,False,6,11,-9.272001,149.022760,1.527804,24,00N_140E,b'\x01\x01\x00\x00\x00\x901w-!\x80b@\xaa\xf1\x...


In [12]:
from burned_embedder.data import calculate_duration_days

In [13]:
# Load the data
df = pd.read_parquet(root_path / "data/processed/radd/southeast_asia_negative_clean.parquet")

# Calculate duration days using the new function
df['duration_days'] = calculate_duration_days(df)

# Display the first few rows with the new column
print("DataFrame with calculated duration_days:")
print(df[['earliest_alert', 'latest_alert', 'duration_days']].head())

# Show some statistics
print(f"\nDuration statistics:")
print(f"Min duration: {df['duration_days'].min()} days")
print(f"Max duration: {df['duration_days'].max()} days")
print(f"Mean duration: {df['duration_days'].mean():.1f} days")
print(f"Median duration: {df['duration_days'].median()} days")

DataFrame with calculated duration_days:
  earliest_alert latest_alert  duration_days
0     2020-09-24   2020-10-19             25
1     2025-07-27   2025-08-24             28
2     2023-08-10   2023-09-03             24
3     2023-11-02   2023-11-14             12
4     2025-08-17   2025-09-10             24

Duration statistics:
Min duration: 12 days
Max duration: 29 days
Mean duration: 22.2 days
Median duration: 24.0 days


In [14]:
import numpy